In [1]:
using DynamicalSystems
using DifferentialEquations
using OrdinaryDiffEq

using StaticArrays

x1 = y1 = z1 = x2 = y2 = z3 = -10..10
box = x1 × y1 × z1 × x2 × y2 × z3

using CairoMakie

In [2]:
function sigma(x)
    return @fastmath 1.0 / ( 1.0 + exp( -10.0 * ( x  - ( - 0.25 ) ) ) )
end

function HR(u, p, t)
        
    a, b, c, d, s, xr, r,  I, vs, k1, k2, el_link  = p
    x1, y1, z1, x2, y2, z2 = u
    
    du1 = y1 + b * x1 ^ 2 - a * x1 ^3 - z1 + I - k1 * ( x1 - vs ) * sigma(x2) + el_link * ( x2 - x1 )
    du2 = c - d * x1 ^2 - y1
    du3 = r * ( s * ( x1 - xr ) - z1 )
    
    du4 = y2 + b * x2 ^ 2 - a * x2 ^3 - z2 + I - k2 * ( x2 - vs ) * sigma(x1) + el_link * ( x1 - x2 )
    du5 = c - d * x2 ^2 - y2
    du6 = r * ( s * ( x2 - xr ) - z2 )
    return SVector(du1, du2, du3,
                    du4, du5, du6)
end

HR (generic function with 1 method)

In [3]:
tspan = (0.0, 300000.0)

condition1 = SA[0.02, 0.01, 0.01, 0.01, 0.01, 0.01]
condition2 = SA[0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

a = 1.
b = 3.
c = 1.
d = 5.

xr = -1.6
r = 0.01

s = 5.
I = 4.
xv = 2.

k1= -0.17; k2 = -0.17

k = 0.0

p = SA[a, b, c, d, s, xr, r, I, xv, k1, k2, k]

12-element SVector{12, Float64} with indices SOneTo(12):
  1.0
  3.0
  1.0
  5.0
  5.0
 -1.6
  0.01
  4.0
  2.0
 -0.17
 -0.17
  0.0

In [33]:
k_space = range(0.0, 1.0, step = 0.01)

0.0:0.01:1.0

In [34]:
for (i, k) in enumerate(k_space)
    if i == 1
        global conditionchaos =   SA[-1.5, 0.0, 0.0, -2.5, 0.0, 0.0]
        global conditionquasi =  SA[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
    end
    println("Initial condition chaos: $conditionchaos"); flush(stdout)
    println("Initial condition quasi: $conditionquasi"); flush(stdout)
    println("k: $k"); flush(stdout)
    
    p = SA[a, b, c, d,
        s, xr, r, I, xv, k1, k2, k]
    
    dschaos = ContinuousDynamicalSystem(HR, conditionchaos, p )
    trchaos = trajectory(dschaos, 300000.0, diffeq = (alg = AutoVern9(Rodas5()),
                                                abstol = 1e-11, reltol = 1e-11, dense = false,  maxiters = 10000000) )
    
    dsquasi = ContinuousDynamicalSystem(HR, conditionquasi, p )
    trcquasi = trajectory(dsquasi, 300000.0, diffeq = (alg = AutoVern9(Rodas5()),
                                                abstol = 1e-11, reltol = 1e-11, dense = false,  maxiters = 10000000) )
    
    
    conditionchaos = trchaos[end, :]
    conditionquasi = trcquasi[end, :]
    
    println("Last point chaos: ", conditionchaos); flush(stdout)
    println("Last point quasi: ", conditionquasi); flush(stdout)
    
    xc, yc, zc  = trchaos[20000000:20030000, 1],  trchaos[20000000:20030000, 2],   trchaos[20000000:20030000, 3]
    xq, yq, zq  = trcquasi[20000000:20030000, 1], trcquasi[20000000:20030000, 2],  trcquasi[20000000:20030000, 3]
    
    fig = Figure()
    ax = Axis3(fig[1, 1])
    lines!(xc, yc, zc, color = "red", linewidth = 0.8)
    lines!(xq, yq, zq, color = "blue", linewidth = 1.5)

    fig
    
    filename = "$(lpad(i, 5, "0")).png"
    directory = "C:\\Users\\olenin.s\\Desktop\\dynamical-systems\\Hindmarrh Rose\\attractors\\res\\"
    directory = directory * filename
    save(directory, fig)
    
    
    
    println(">>>>>>>>>>>>>>>")
    println("")
        
end

Initial condition chaos: [-1.5, 0.0, 0.0, -2.5, 0.0, 0.0]
Initial condition quasi: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
k: 0.0
Last point chaos: [-0.49898797140841383, -1.041107136841029, 3.6458528048946643, -0.9330464983989869, -3.458236478969528, 3.920040824803387]
Last point quasi: [-1.329189856339424, -7.770776061899801, 3.8807617543263975, -1.329189856339424, -7.770776061899801, 3.8807617543263975]
>>>>>>>>>>>>>>>

Initial condition chaos: [-0.49898797140841383, -1.041107136841029, 3.6458528048946643, -0.9330464983989869, -3.458236478969528, 3.920040824803387]
Initial condition quasi: [-1.329189856339424, -7.770776061899801, 3.8807617543263975, -1.329189856339424, -7.770776061899801, 3.8807617543263975]
k: 0.01
Last point chaos: [-0.9576930957750247, -3.8381896089679075, 3.8218980021286506, -1.1263523559071282, -5.153137314304569, 4.098543275665978]
Last point quasi: [-0.8880048737587348, -3.1980738116223617, 3.834237658429783, -0.8880048737587348, -3.1980738116223617, 3.834237658429783

Initial condition quasi: [-1.178223932065377, -5.759767017796963, 4.054953561914162, -1.178223932065377, -5.759767017796963, 4.054953561914162]
k: 0.14
Last point chaos: [-0.7774061917877026, -2.141360619147437, 4.0774753294821995, -1.1142696491877713, -4.749392466559684, 4.4448842812325156]
Last point quasi: [-1.1596634023939931, -6.014917353684204, 3.5516212560257294, -1.1596634023939931, -6.014917353684204, 3.5516212560257294]
>>>>>>>>>>>>>>>

Initial condition chaos: [-0.7774061917877026, -2.141360619147437, 4.0774753294821995, -1.1142696491877713, -4.749392466559684, 4.4448842812325156]
Initial condition quasi: [-1.1596634023939931, -6.014917353684204, 3.5516212560257294, -1.1596634023939931, -6.014917353684204, 3.5516212560257294]
k: 0.15
Last point chaos: [-1.1189188455527632, -5.619901428601079, 3.5599331527578135, -0.6817537257087427, -2.09362710072226, 3.4239538799316893]
Last point quasi: [-0.9105923225873406, -3.1595948834619287, 4.079922302436023, -0.9105923225873406, -3.1

>>>>>>>>>>>>>>>

Initial condition chaos: [-1.6048129758846774, -11.622544046875346, 4.112728811535289, 0.08000133031992289, 0.7595925556749429, 3.9753157215245496]
Initial condition quasi: [-1.2276850183480927, -6.629466972555574, 3.7332294494699263, -1.2276850183480927, -6.629466972555574, 3.7332294494699263]
k: 0.28
Last point chaos: [-0.46771060496791683, -0.733563075768399, 3.618251885466243, -1.390437532365177, -8.174558228816391, 4.547235127347093]
Last point quasi: [-0.9237142939214771, -3.5536784255314644, 3.758455206170792, -0.9237142939214771, -3.5536784255314644, 3.758455206170792]
>>>>>>>>>>>>>>>

Initial condition chaos: [-0.46771060496791683, -0.733563075768399, 3.618251885466243, -1.390437532365177, -8.174558228816391, 4.547235127347093]
Initial condition quasi: [-0.9237142939214771, -3.5536784255314644, 3.758455206170792, -0.9237142939214771, -3.5536784255314644, 3.758455206170792]
k: 0.29
Last point chaos: [-1.536199686909002, -10.838016333207174, 3.770365897470498, -

Last point quasi: [-0.6444553403312308, -1.7813922999252954, 3.598339787433575, -0.6444553403312308, -1.7813922999252954, 3.598339787433575]
>>>>>>>>>>>>>>>

Initial condition chaos: [0.02179549978486653, 0.8281809515728363, 4.468316614616916, -0.04009540306894779, -0.5560645465900129, 4.3349493079029235]
Initial condition quasi: [-0.6444553403312308, -1.7813922999252954, 3.598339787433575, -0.6444553403312308, -1.7813922999252954, 3.598339787433575]
k: 0.42
Last point chaos: [-0.19836104284224357, 0.8771099048701203, 4.795226364388579, 0.0018259455668325926, 0.9441389096801565, 4.552152446374133]
Last point quasi: [-0.8879393541894699, -3.050764581450324, 3.9994611400470426, -0.8879393541894699, -3.050764581450324, 3.9994611400470426]
>>>>>>>>>>>>>>>

Initial condition chaos: [-0.19836104284224357, 0.8771099048701203, 4.795226364388579, 0.0018259455668325926, 0.9441389096801565, 4.552152446374133]
Initial condition quasi: [-0.8879393541894699, -3.050764581450324, 3.9994611400470426, -

k: 0.55
Last point chaos: [-0.5644830937833956, -0.44535133118106535, 4.648749007945016, -0.18014577229763376, 0.6354062057418663, 4.422617018059682]
Last point quasi: [-0.2624666724274197, 0.04653193305343474, 3.8928155475789135, -0.2624666724274197, 0.04653193305343474, 3.8928155475789135]
>>>>>>>>>>>>>>>

Initial condition chaos: [-0.5644830937833956, -0.44535133118106535, 4.648749007945016, -0.18014577229763376, 0.6354062057418663, 4.422617018059682]
Initial condition quasi: [-0.2624666724274197, 0.04653193305343474, 3.8928155475789135, -0.2624666724274197, 0.04653193305343474, 3.8928155475789135]
k: 0.56
Last point chaos: [-1.4961994217071397, -9.950716367857629, 4.113731543872825, -1.5222580777057664, -10.106684819602766, 4.414349087816067]
Last point quasi: [-0.8446948129526307, -2.5426217469287096, 4.283864102318206, -0.8446948129526307, -2.5426217469287096, 4.283864102318206]
>>>>>>>>>>>>>>>

Initial condition chaos: [-1.4961994217071397, -9.950716367857629, 4.113731543872825,

Initial condition quasi: [-0.8129145990745074, -2.9245330355897314, 3.5069908957554143, -0.8129145990745074, -2.9245330355897314, 3.5069908957554143]
k: 0.69
Last point chaos: [-0.8465025203101538, -2.4408921615810724, 4.332761330709277, -0.8465025203101538, -2.4408921615810724, 4.332761330709277]
Last point quasi: [-0.8574779403507581, -2.7786764272249784, 4.0424870185297435, -0.8574779403507581, -2.7786764272249784, 4.0424870185297435]
>>>>>>>>>>>>>>>

Initial condition chaos: [-0.8465025203101538, -2.4408921615810724, 4.332761330709277, -0.8465025203101538, -2.4408921615810724, 4.332761330709277]
Initial condition quasi: [-0.8574779403507581, -2.7786764272249784, 4.0424870185297435, -0.8574779403507581, -2.7786764272249784, 4.0424870185297435]
k: 0.7
Last point chaos: [-1.2384128884910286, -6.734837317712116, 3.758438404547289, -1.2384128884910286, -6.734837317712116, 3.758438404547289]
Last point quasi: [-1.13688664617891, -5.260762433280345, 4.103409528099133, -1.13688664617891, -

>>>>>>>>>>>>>>>

Initial condition chaos: [-0.5758805575392814, -1.0760000982174622, 4.010321506461268, -0.5758805575392814, -1.0760000982174622, 4.010321506461268]
Initial condition quasi: [-0.2511445160226115, 0.020332293995836533, 3.81497005918648, -0.2511445160226115, 0.020332293995836533, 3.81497005918648]
k: 0.83
Last point chaos: [-1.217244971392773, -6.153298067875458, 4.11526914660123, -1.217244971392773, -6.153298067875458, 4.11526914660123]
Last point quasi: [-0.806849190570786, -2.292509639722598, 4.339164127273123, -0.806849190570786, -2.292509639722598, 4.339164127273123]
>>>>>>>>>>>>>>>

Initial condition chaos: [-1.217244971392773, -6.153298067875458, 4.11526914660123, -1.217244971392773, -6.153298067875458, 4.11526914660123]
Initial condition quasi: [-0.806849190570786, -2.292509639722598, 4.339164127273123, -0.806849190570786, -2.292509639722598, 4.339164127273123]
k: 0.84
Last point chaos: [-0.7802722211973745, -2.626642505903576, 3.5874760953869593, -0.7802722211973

Last point quasi: [-0.6657270927850539, -1.5588688340494528, 3.999133656796127, -0.6657270927850539, -1.5588688340494528, 3.999133656796127]
>>>>>>>>>>>>>>>

Initial condition chaos: [-1.0014138410064748, -3.8224292717300736, 4.209911774352224, -1.0014138410064748, -3.8224292717300736, 4.209911774352224]
Initial condition quasi: [-0.6657270927850539, -1.5588688340494528, 3.999133656796127, -0.6657270927850539, -1.5588688340494528, 3.999133656796127]
k: 0.97
Last point chaos: [-1.316213817232748, -7.735335649382492, 3.7349080863071547, -1.316213817232748, -7.735335649382492, 3.7349080863071547]
Last point quasi: [-1.1906343657400755, -5.819830457493881, 4.142625328176635, -1.1906343657400755, -5.819830457493881, 4.142625328176635]
>>>>>>>>>>>>>>>

Initial condition chaos: [-1.316213817232748, -7.735335649382492, 3.7349080863071547, -1.316213817232748, -7.735335649382492, 3.7349080863071547]
Initial condition quasi: [-1.1906343657400755, -5.819830457493881, 4.142625328176635, -1.19063436

In [30]:
using FFMPEG

In [35]:
framerate = 3
directory = "C:\\Users\\olenin.s\\Desktop\\dynamical-systems\\Hindmarrh Rose\\attractors\\res\\"
gifname = "output.gif"
gifname = directory * gifname

"C:\\Users\\olenin.s\\Desktop\\dynamical-systems\\Hindmarrh Rose\\attractors\\res\\output.gif"

In [36]:
FFMPEG.ffmpeg_exe(`-framerate $(framerate) -f image2 -i $(directory)/%5d.png -vf "scale=1200:900" -c:v libx264 -pix_fmt yuv420p -y "C:\\Users\\olenin.s\\Desktop\\dynamical-systems\\Hindmarrh Rose\\attractors\\res\\video.mov"`)

ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 8.1.0 (GCC)
  configuration: --enable-cross-compile --cross-prefix=/opt/x86_64-w64-mingw32/bin/x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw64 --cc=cc --cxx=c++ --dep-cc=cc --ar=ar --nm=nm --sysinclude=/workspace/destdir/include --pkg-config=/usr/bin/pkg-config --pkg-config-flags=--static --prefix=/workspace/destdir --sysroot=/opt/x86_64-w64-mingw32/x86_64-w64-mingw32/sys-root --extra-libs=-lpthread --enable-gpl --enable-version3 --enable-nonfree --disable-static --enable-shared --enable-pic --disable-debug --disable-doc --enable-avresample --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libvorbis --enable-libx264 --enable-libx265 --enable-libvpx --enable-encoders --enable-decoders --enable-muxers --enable-demuxers --enable-parsers --enable-openssl --disable-schannel --extra-cflags=-I/workspace/destdir/include --extra-ldflags=-

Process(`'C:\Users\olenin.s\.julia\artifacts\4bb7d8b702e81f58099ea3a710188a616999662b\bin\ffmpeg.exe' -framerate 3 -f image2 -i 'C:\Users\olenin.s\Desktop\dynamical-systems\Hindmarrh Rose\attractors\res\/%5d.png' -vf scale=1200:900 -c:v libx264 -pix_fmt yuv420p -y 'C:\Users\olenin.s\Desktop\dynamical-systems\Hindmarrh Rose\attractors\res\video.mov'`, ProcessExited(0))